In [1]:
pip install arxiv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [5]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [7]:
wiki.name

'wikipedia'

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [1]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith_search",
                                       "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

NameError: name 'retriever' is not defined

In [2]:
retriever_tool.name

NameError: name 'retriever_tool' is not defined

In [ ]:
##Arxiv tool

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper = arxiv_wrapper)
arxiv.name

In [ ]:
tools=[[wiki,arxiv,retriever_tool]]

In [ ]:
from dotenv import load_dotenv

load_dotenv()

import os
os.environ['OPENAI_APAI_KEY']=os.getenv('OPENAI_API_KEY')
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.55-turbo-0125",temperature=0)

In [ ]:
from langchain import hub
#Get the prompt to use - ou can modify this
prompt = hub.pull("hwchasel177/openai-functions-agent")
prompt.messages

In [ ]:
##Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [ ]:
#Agents Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})